In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pingouin as pg
import sys

# Print version info
print(f"Python version: {sys.version}")
print(f"Pingouin version: {pg.__version__}")

# Directory setup
root_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
data_dir = os.path.join(root_dir, 'data')

Python version: 3.12.9 (main, Mar 18 2025, 16:22:10) [Clang 16.0.0 (clang-1600.0.26.6)]
Pingouin version: 0.5.5


In [3]:
# Statistical analysis: Paired samples t-tests by section

# Load data
df_master = pd.read_csv(os.path.join(data_dir, 'mastersheet.csv'))

# Label mapping
df_master['section'] = df_master['section'].map({'cervical': 'Cervical', 
                                       'lumbar': 'Lumbar'})
df_master['condition'] = df_master['condition'].map({'baseline': 'SatPad–',
                                           'satpads': 'SatPad+'})

variables = [
    ('tsnr_cord', 'tSNR'),
    ('cv', 'Timeseries CoV'),
    ('dvars_nstd', 'DVARS'),
    ('fd', 'FD'),
    ('sd_fieldmap', 'Magnetic field SD'),
    ('n_ghosted_slices', 'Number of ghosted slices')
]

# Store results for summary table
results = []

for section in ['Cervical', 'Lumbar']:
    
    # Filter data
    section_data = df_master[df_master['section'] == section]
    
    for var_name, display_name in variables:

        # Create paired data by subject
        pivot_data = section_data.pivot_table(
            index='id', 
            columns='condition', 
            values=var_name, 
            aggfunc='first'
        ).dropna()
            
        satpad_minus = pivot_data['SatPad–']
        satpad_plus = pivot_data['SatPad+']
        
        # Run paired t-test
        ttest_results = pg.ttest(satpad_minus, satpad_plus, paired=True)
        t_stat = ttest_results['T'].iloc[0]
        p_value = ttest_results['p-val'].iloc[0]
        
        # Calculate descriptive statistics
        mean_minus = satpad_minus.mean()
        std_minus = satpad_minus.std()
        mean_plus = satpad_plus.mean()
        std_plus = satpad_plus.std()
        cv_minus = (std_minus / mean_minus) * 100
        cv_plus = (std_plus / mean_plus) * 100
        n_pairs = len(satpad_minus)
        
        # Calculate difference scores
        differences = satpad_minus - satpad_plus
        mean_diff = differences.mean()
        std_diff = differences.std()
        
        # Calculate percentage signal change based on group means: ((Mean_SatPad+ - Mean_SatPad–) / Mean_SatPad–) * 100
        group_percent_change = np.round(((mean_plus - mean_minus) / mean_minus) * 100)

         # Determine significance
        if p_value < 0.001:
            sig_level = "***"
        elif p_value < 0.01:
            sig_level = "**"
        elif p_value < 0.05:
            sig_level = "*"
        else:
            sig_level = "ns"
        
        # Effect size (Hedges' g for paired samples)
        eff_size = pg.compute_effsize(satpad_minus, satpad_plus, paired=True, eftype='hedges')

        # Get confidence intervals 
        ci_lower = ttest_results['CI95%'].iloc[0][0]
        ci_upper = ttest_results['CI95%'].iloc[0][1]

        # Store results
        results.append({
            'Section': section,
            'Variable': display_name,
            'N_pairs': n_pairs,
            'SatPad-_Mean': mean_minus,
            'SatPad-_SD': std_minus,
            'SatPad+_Mean': mean_plus,
            'SatPad+_SD': std_plus,
            'Mean_Diff': mean_diff,
            'SD_Diff': std_diff,
            'SatPad-_CV': cv_minus,
            'SatPad+_CV': cv_plus,
            'CI_Lower': ci_lower,
            'CI_Upper': ci_upper,
            'Percent_Change': group_percent_change,
            't_statistic': t_stat,
            'p_value': p_value,
            'Significance': sig_level,
            'Hedges_g': eff_size
        })

# Create summary table 
results_df = pd.DataFrame(results)

# Multiple comparisons correction (Benjamini-Hochberg FDR)
results_df['p_value_fdr'] = np.nan
results_df['Significance_fdr'] = 'ns'

for section in ['Cervical', 'Lumbar']:
    section_mask = results_df['Section'] == section
    section_pvals = results_df.loc[section_mask, 'p_value'].values
    
    if len(section_pvals) > 0:
        reject, pvals_corrected = pg.multicomp(section_pvals, method='fdr_bh')
        results_df.loc[section_mask, 'p_value_fdr'] = pvals_corrected
        results_df.loc[section_mask, 'Significance_fdr'] = [
            '***' if p < 0.001 else '**' if p < 0.01 else '*' if p < 0.05 else 'ns' 
            for p in pvals_corrected
        ]

# Summary of significant results
print("\n" + "=" * 60)
print("SUMMARY OF SIGNIFICANT RESULTS BY SECTION")
print("=" * 60)

significant_results_main = results_df[results_df['Significance_fdr'] != 'ns']
if len(significant_results_main) > 0:
    for _, row in significant_results_main.iterrows():
        effect_size_desc = "small" if abs(row['Hedges_g']) < 0.5 else ("medium" if abs(row['Hedges_g']) < 0.8 else "large")
        direction = "increase" if row['Percent_Change'] > 0 else "decrease"
        print(f"{row['Section']} {row['Variable']}: {row['Percent_Change']:.0f}% {direction} (t({row['N_pairs']-1})={row['t_statistic']:.2f}, p-FDR={row['p_value_fdr']:.3f}, 95% CI [{row['CI_Lower']:.2f}, {row['CI_Upper']:.2f}], Hedges g={row['Hedges_g']:.2f}, {effect_size_desc} effect size)")
else:
    print("No significant differences found in the overall analysis.")

# Display the summary table
print("\n\n" + "=" * 100)
print("SUMMARY TABLE - PAIRED SAMPLES T-TESTS BY SECTION")
print("=" * 100)

# Reorder columns for better readability
column_order = ['Section', 'Variable', 'N_pairs', 
                'Percent_Change',
                't_statistic', 'p_value', 'Significance', 
                'p_value_fdr', 'Significance_fdr',
                'SatPad-_Mean', 'SatPad-_SD', 
                'SatPad+_Mean', 'SatPad+_SD', 
                'Mean_Diff', 'SD_Diff', 
                'SatPad-_CV', 'SatPad+_CV',
                'CI_Lower', 'CI_Upper',
                'Hedges_g'
                ]
results_df = results_df[column_order]
print(results_df.round(3).to_string(index=False))

print("\n\nSignificance levels: *** p<0.001, ** p<0.01, * p<0.05, ns = not significant")
print("FDR correction applied (Benjamini-Hochberg method)")
print("Hedges' g interpretation (paired): |g|<0.2 (small), |g|<0.5 (medium), |g|≥0.8 (large)")
print("CI_Lower/CI_Upper: 95% confidence interval bounds for the mean difference")
print("Positive Mean_Diff indicates SatPad– > SatPad+; Negative indicates SatPad– < SatPad+")
print("Percent_Change calculated from group means: ((Mean_SatPad+ - Mean_SatPad–) / Mean_SatPad–) × 100")
print("Positive Percent_Change indicates improvement (increase) with SatPad+; Negative indicates decrease with SatPad+")


SUMMARY OF SIGNIFICANT RESULTS BY SECTION
Cervical tSNR: 14% increase (t(9)=-2.78, p-FDR=0.043, 95% CI [-7.72, -0.79], Hedges g=-1.15, large effect size)
Cervical Magnetic field SD: -25% decrease (t(9)=9.02, p-FDR=0.000, 95% CI [46.94, 78.38], Hedges g=2.26, large effect size)
Cervical Number of ghosted slices: -86% decrease (t(9)=3.50, p-FDR=0.020, 95% CI [0.85, 3.95], Hedges g=1.05, large effect size)


SUMMARY TABLE - PAIRED SAMPLES T-TESTS BY SECTION
 Section                 Variable  N_pairs  Percent_Change  t_statistic  p_value Significance  p_value_fdr Significance_fdr  SatPad-_Mean  SatPad-_SD  SatPad+_Mean  SatPad+_SD  Mean_Diff  SD_Diff  SatPad-_CV  SatPad+_CV  CI_Lower  CI_Upper  Hedges_g
Cervical                     tSNR       10            14.0       -2.778    0.021            *        0.043                *        30.127       4.415        34.382       2.368     -4.254    4.843      14.654       6.887     -7.72     -0.79    -1.150
Cervical           Timeseries CoV       

In [4]:
# Statistical analysis: Paired samples t-tests for segmental tSNR data

# Load data
df_tsnr_segmental = pd.read_csv(os.path.join(data_dir, 'segmental_plots.csv'))

# Label mapping 
df_tsnr_segmental['section'] = df_tsnr_segmental['section'].map({'cervical': 'Cervical', 
                                                               'lumbar': 'Lumbar'})
df_tsnr_segmental['condition'] = df_tsnr_segmental['condition'].map({'baseline': 'SatPad–',
                                                                   'satpads': 'SatPad+'})

# Store results 
segmental_results = []

for section in ['Cervical', 'Lumbar']:
    
    # Filter data 
    section_data = df_tsnr_segmental[df_tsnr_segmental['section'] == section]
    
    # Get unique segments 
    segments = sorted(section_data['segment'].unique(), key=lambda x: int(x[1:]))
    
    for segment in segments:
        
        # Filter data 
        segment_data = section_data[section_data['segment'] == segment]
        
        # Create paired data by subject
        pivot_data = segment_data.pivot_table(
            index='id', 
            columns='condition', 
            values='tsnr', 
            aggfunc='first'
        ).dropna()
            
        satpad_minus = pivot_data['SatPad–']
        satpad_plus = pivot_data['SatPad+']
        
        # Run paired t-test 
        ttest_results = pg.ttest(satpad_minus, satpad_plus, paired=True)
        t_stat = ttest_results['T'].iloc[0]
        p_value = ttest_results['p-val'].iloc[0]
        
        # Calculate descriptive statistics
        mean_minus = satpad_minus.mean()
        std_minus = satpad_minus.std()
        mean_plus = satpad_plus.mean()
        std_plus = satpad_plus.std()
        n_pairs = len(satpad_minus)
        
        # Calculate difference scores
        differences = satpad_minus - satpad_plus
        mean_diff = differences.mean()
        std_diff = differences.std()
        
        # Calculate percentage signal change based on group means: ((Mean_SatPad+ - Mean_SatPad–) / Mean_SatPad–) * 100
        group_percent_change = np.round(((mean_plus - mean_minus) / mean_minus) * 100)

        # Determine significance
        if p_value < 0.001:
            sig_level = "***"
        elif p_value < 0.01:
            sig_level = "**"
        elif p_value < 0.05:
            sig_level = "*"
        else:
            sig_level = "ns"

        # Effect size (Hedges' g for paired samples)
        eff_size = pg.compute_effsize(satpad_minus, satpad_plus, paired=True, eftype='hedges')

        # Get confidence intervals 
        ci_lower = ttest_results['CI95%'].iloc[0][0]
        ci_upper = ttest_results['CI95%'].iloc[0][1]
        
        # Store results
        segmental_results.append({
            'Section': section,
            'Segment': segment.upper(),
            'N_pairs': n_pairs,
            'SatPad-_Mean': mean_minus,
            'SatPad-_SD': std_minus,
            'SatPad+_Mean': mean_plus,
            'SatPad+_SD': std_plus,
            'Mean_Diff': mean_diff,
            'SD_Diff': std_diff,
            'CI_Lower': ci_lower,
            'CI_Upper': ci_upper,
            'Percent_Change': group_percent_change,
            't_statistic': t_stat,
            'p_value': p_value,
            'Significance': sig_level,
            'Hedges_g': eff_size
        })

# Create summary table for segmental results FIRST before using it
segmental_results_df = pd.DataFrame(segmental_results)

# Multiple comparisons correction (Benjamini-Hochberg FDR) - SEPARATELY for each section
segmental_results_df['p_value_fdr'] = np.nan
segmental_results_df['Significance_fdr'] = 'ns'

for section in ['Cervical', 'Lumbar']:
    section_mask = segmental_results_df['Section'] == section
    section_pvals = segmental_results_df.loc[section_mask, 'p_value'].values
    
    if len(section_pvals) > 0:
        reject, pvals_corrected = pg.multicomp(section_pvals, method='fdr_bh')
        segmental_results_df.loc[section_mask, 'p_value_fdr'] = pvals_corrected
        segmental_results_df.loc[section_mask, 'Significance_fdr'] = [
            '***' if p < 0.001 else '**' if p < 0.01 else '*' if p < 0.05 else 'ns' 
            for p in pvals_corrected
        ]

# Summary of significant results
print("\n" + "=" * 60)
print("SUMMARY OF SIGNIFICANT SEGMENTAL RESULTS")
print("=" * 60)

significant_results = segmental_results_df[segmental_results_df['Significance_fdr'] != 'ns']
if len(significant_results) > 0:
    for _, row in significant_results.iterrows():
        effect_size_desc = "small" if abs(row['Hedges_g']) < 0.5 else ("medium" if abs(row['Hedges_g']) < 0.8 else "large")
        direction = "increase" if row['Percent_Change'] > 0 else "decrease"
        print(f"{row['Section']} {row['Segment']}: {row['Percent_Change']:.0f}% {direction} (t({row['N_pairs']-1})={row['t_statistic']:.2f}, p-FDR={row['p_value_fdr']:.3f}, 95% CI [{row['CI_Lower']:.2f}, {row['CI_Upper']:.2f}], Hedges g={row['Hedges_g']:.2f}, {effect_size_desc} effect size)")
else:
    print("No significant differences found in segmental tSNR analysis.")

# Display the summary table
print("\n\n" + "=" * 120)
print("SUMMARY TABLE - PAIRED SAMPLES T-TESTS FOR SEGMENTAL tSNR")
print("=" * 120)

# Reorder columns for better readability
column_order = ['Section', 'Segment', 'N_pairs', 
                'Percent_Change',
                't_statistic', 'p_value', 'Significance', 
                'p_value_fdr', 'Significance_fdr',
                'SatPad-_Mean', 'SatPad-_SD', 
                'SatPad+_Mean', 'SatPad+_SD', 
                'Mean_Diff', 'SD_Diff', 
                'CI_Lower', 'CI_Upper',
                'Hedges_g'
                ]
segmental_results_df = segmental_results_df[column_order]
print(segmental_results_df.round(3).to_string(index=False))

print("\n\nSignificance levels: *** p<0.001, ** p<0.01, * p<0.05, ns = not significant")
print("FDR correction applied (Benjamini-Hochberg method)")
print("Hedges' g interpretation (paired): |g|<0.2 (small), |g|<0.5 (medium), |g|≥0.8 (large)")
print("CI_Lower/CI_Upper: 95% confidence interval bounds for the mean difference")
print("Positive Mean_Diff indicates SatPad– > SatPad+; Negative indicates SatPad– < SatPad+")
print("Percent_Change calculated from group means: ((Mean_SatPad+ - Mean_SatPad–) / Mean_SatPad–) × 100")
print("Positive Percent_Change indicates improvement (increase) with SatPad+; Negative indicates decrease with SatPad+")


SUMMARY OF SIGNIFICANT SEGMENTAL RESULTS
Cervical C7: 17% increase (t(9)=-5.38, p-FDR=0.004, 95% CI [-7.56, -3.09], Hedges g=-1.02, large effect size)


SUMMARY TABLE - PAIRED SAMPLES T-TESTS FOR SEGMENTAL tSNR
 Section Segment  N_pairs  Percent_Change  t_statistic  p_value Significance  p_value_fdr Significance_fdr  SatPad-_Mean  SatPad-_SD  SatPad+_Mean  SatPad+_SD  Mean_Diff  SD_Diff  CI_Lower  CI_Upper  Hedges_g
Cervical      C1       10            15.0       -1.537    0.159           ns        0.254               ns        36.394      10.208        41.980       5.733     -5.586   11.492    -13.81      2.63    -0.646
Cervical      C2       10             6.0       -1.082    0.307           ns        0.410               ns        39.565       7.184        41.775       5.145     -2.210    6.458     -6.83      2.41    -0.339
Cervical      C3       10            24.0       -2.375    0.042            *        0.111               ns        30.585       9.334        37.936       4.110   

In [5]:
# Statistical analysis: Inter-rater reliability for number of ghosted slices

# Load data
df_ghosting = pd.read_csv(os.path.join(data_dir, 'ghosting.csv'))

# Create a unique identifier for each observation (subject + condition)
df_ghosting['observation'] = df_ghosting['id'] + '_' + df_ghosting['condition']

# Reshape from wide to long format 
df_icc = pd.melt(
    df_ghosting,
    id_vars=['observation'],
    value_vars=['n_ghosted_slices_osk', 'n_ghosted_slices_djl'],
    var_name='rater',
    value_name='n_ghosted_slices'
)

# Clean up rater names
df_icc['rater'] = df_icc['rater'].str.replace('n_ghosted_slices_', '')

# Calculate ICC
icc_results = pg.intraclass_corr(
    data=df_icc,
    targets='observation',  # Each unique measurement
    raters='rater',         # osk vs djl
    ratings='n_ghosted_slices'
)

# Calculate ICC3 (two-way mixed, single measures)
icc3 = icc_results[icc_results['Type'] == 'ICC3']

# Display ICC3 results
print("\n" + "=" * 60)
print("INTER-RATER RELIABILITY FOR NUMBER OF GHOSTED SLICES")
print("=" * 60)
print(f"\nICC(3,1) = {icc3['ICC'].values[0]:.3f}")
print("\n\nICC interpretation: " \
"\n         <0.4        poor, " \
"\n         0.4-0.59    fair, " \
"\n         0.75-0.9    good, " \
"\n         >0.9        excellent.")


INTER-RATER RELIABILITY FOR NUMBER OF GHOSTED SLICES

ICC(3,1) = 0.749


ICC interpretation: 
         <0.4        poor, 
         0.4-0.59    fair, 
         0.75-0.9    good, 
         >0.9        excellent.
